## A taste of MLJ

### Baby steps

Let's load a reduced version of the well-known Ames House Price data set (containing six of the more important categorical features and six of the more important numerical features). We'll extract the data from one of the built-in [tasks](https://alan-turing-institute.github.io/MLJ.jl/dev/working_with_tasks/) (not discussed in this notebook):

In [1]:
using Pkg

Pkg.activate(@__DIR__)
Pkg.instantiate()

using MLJ
using DataFrames, Statistics

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


In [2]:
task = load_reduced_ames()
X, y = task()

train, test = partition(eachindex(y), 0.70, shuffle=true); # 70:30 split

first(X, 4)

,OverallQual,GrLivArea,Neighborhood,x1stFlrSF,TotalBsmtSF,BsmtFinSF1,LotArea
,Float64,Float64,Categorical…,Float64,Float64,Float64,Float64
1,5.0,816.0,Mitchel,816.0,816.0,816.0,6600.0
2,8.0,2028.0,Timber,2028.0,1868.0,1460.0,11443.0
3,7.0,1509.0,Gilbert,807.0,783.0,0.0,7875.0
4,6.0,1686.0,NWAmes,1686.0,1686.0,625.0,10240.0


A *model* is a container for hyperparameters:

In [3]:
constant_model = ConstantRegressor()

ConstantRegressor(distribution_type = Distributions.Normal,) @ 1…45

Wrapping the model in data creates a *machine* which will store training outcomes (called *fit-results*):

In [4]:
constant = machine(constant_model, X, y)

Machine{ConstantRegressor} @ 1…85


Training and making a new prediction:

In [5]:
fit!(constant, rows=train)
yhat = predict(constant, X[test,:]);
yhat[1:4]

┌ Info: Training Machine{ConstantRegressor} @ 1…85.
└ @ MLJ /Users/anthony/.julia/packages/MLJ/w0rRg/src/machines.jl:102
┌ Info: Fitted a constant probability distribution, Distributions.Normal{Float64}(μ=179790.87340529932, σ=76344.53307783141).
└ @ MLJ.Constant /Users/anthony/.julia/packages/MLJ/w0rRg/src/builtins/Constant.jl:45


4-element Array{Distributions.Normal{Float64},1}:
 Distributions.Normal{Float64}(μ=179790.87340529932, σ=76344.53307783141)
 Distributions.Normal{Float64}(μ=179790.87340529932, σ=76344.53307783141)
 Distributions.Normal{Float64}(μ=179790.87340529932, σ=76344.53307783141)
 Distributions.Normal{Float64}(μ=179790.87340529932, σ=76344.53307783141)

The model we chose can provide probabilistic predictions and so does so by default.

In [6]:
mean.(yhat)[1:4]

4-element Array{Float64,1}:
 179790.87340529932
 179790.87340529932
 179790.87340529932
 179790.87340529932

In [7]:
rmsl(yhat, y[test]) # applies root-mean-square-log loss to prediction means

0.4012495878729087

Or, we could have skipped the train/test definitions and evaluated one line:

In [8]:
evaluate!(constant, resampling=Holdout(fraction_train=0.7), measure=rmsl)

┌ Info: Evaluating using a holdout set. 
│ fraction_train=0.7 
│ shuffle=false 
│ measure=MLJ.rmsl 
│ operation=StatsBase.predict 
│ Resampling from all rows. 
└ @ MLJ /Users/anthony/.julia/packages/MLJ/w0rRg/src/resampling.jl:91


0.4095126492179508

### Something fancier

Let's define a composite model type that:

- one-hot encodes the inputs
- log transforms the target
- fits specified K-nearest neighbour and ridge regressor models to the data
- computes a weighted average of individual model predictions
- inverse transforms (exponentiates) the blended predictions

MLJ will eventually have macros for quickly building this kind of composite model (and more sophisticated model stacks). Nevertheless, building the model with our bare hands is easier than in other popular frameworks:

In [9]:
@load RidgeRegressor 
@load KNNRegressor

# the new model struture, with other models as hyperparameters:
mutable struct KNNRidgeBlend <:DeterministicNetwork
    knn_model
    ridge_model
    weight # between 0 and 1
end

┌ Info: A model named "RidgeRegressor" is already loaded.
│ Nothing new loaded. 
└ @ MLJ /Users/anthony/.julia/packages/MLJ/w0rRg/src/loading.jl:163
┌ Info: A model named "KNNRegressor" is already loaded.
│ Nothing new loaded. 
└ @ MLJ /Users/anthony/.julia/packages/MLJ/w0rRg/src/loading.jl:163


Here `DeterministicNetwork` means that our model will make determinisitic (as opposed to probabilistic) predictions, and that its `fit` method returns a node in an MLJ *learning network*: 

In [10]:
# model fitting instructions:
function MLJ.fit(model::KNNRidgeBlend, X, y)
    
    # source nodes of the "learning network" being wrappped:    
    
    Xs = source(X) 
    ys = source(y)

    hot = machine(OneHotEncoder(), Xs)

    # W, z, zhat and yhat are nodes in the network:
    
    W = transform(hot, Xs) # one-hot encode the input
    z = log(ys) # transform the target
    
    ridge_model = model.ridge_model
    knn_model = model.knn_model

    ridge = machine(ridge_model, W, z) 
    knn = machine(knn_model, W, z)

    zhat = model.weight*predict(ridge, W) + (1 - model.weight)*predict(knn, W) # average the predictions of KNN and ridge models
    yhat = exp(zhat) # inverse the target transformation
    
    fit!(yhat, verbosity=0)
    
    return yhat
end


We now instantiate a blended model and evaluate it:

In [11]:
knn_model = KNNRegressor(K=2)
ridge_model = RidgeRegressor(lambda=0.1)
knn_ridge_blend_model = KNNRidgeBlend(knn_model, ridge_model, 0.90)
knn_ridge_blend = machine(knn_ridge_blend_model, X, y)
er = evaluate!(knn_ridge_blend, resampling=Holdout(fraction_train=0.7), measure=rmsl) 

┌ Info: Evaluating using a holdout set. 
│ fraction_train=0.7 
│ shuffle=false 
│ measure=MLJ.rmsl 
│ operation=StatsBase.predict 
│ Resampling from all rows. 
└ @ MLJ /Users/anthony/.julia/packages/MLJ/w0rRg/src/resampling.jl:91


0.13108966715886725

We can inspect the nested hyperparameters of our composite model:

In [12]:
params(knn_ridge_blend_model)

(knn_model = (K = 2,
              metric = MLJ.KNN.euclidean,
              kernel = MLJ.KNN.reciprocal,),
 ridge_model = (lambda = 0.1,),
 weight = 0.9,)

For our next trick, we wrap the model in a tuning strategy, producing a new tuned model. 

The form of `nested_ranges` below matches the pattern above (with parameters not being tuned omitted):

In [13]:
K_range = range(knn_model, :K, lower=2, upper=100, scale=:log10)
lambda_range = range(ridge_model, :lambda, lower= 1e-6, upper=10, scale = :log10)

nested_ranges = (knn_model = (K = K_range,), ridge_model = (lambda = lambda_range,))

tuning = Grid(resolution=12)
resampling = CV(nfolds=6)

tuned_blended_model = TunedModel(model=knn_ridge_blend_model, 
    tuning=tuning, resampling=resampling, nested_ranges=nested_ranges, measure=rmsl)

MLJ.DeterministicTunedModel(model = KNNRidgeBlend{} @ 7…41,
                            tuning = Grid{} @ 1…80,
                            resampling = CV{} @ 2…56,
                            measure = MLJ.rmsl,
                            operation = StatsBase.predict,
                            nested_ranges = (knn_model = (K = NumericRange{K} @ 1…07,), ridge_model = (lambda = NumericRange{lambda} @ 1…94,)),
                            minimize = true,
                            full_report = true,) @ 1…97

Fitting a corresponding machine searches for the model with optimal performance, as determined by the specified tuning/resampling strategy, and trains the best model on all specifed data (in this case on `train`). So, predictions of the fitted machine are predictions using optimal hyperparameters.

In [14]:
tuned_blended = machine(tuned_blended_model, X, y)
fit!(tuned_blended, rows=train);

┌ Info: Training Machine{DeterministicTunedModel} @ 1…10.
└ @ MLJ /Users/anthony/.julia/packages/MLJ/w0rRg/src/machines.jl:102
Iterating over a 144-point grid: 100%[=========================] Time: 0:00:39
┌ Info: Training best model on all supplied data.
└ @ MLJ /Users/anthony/.julia/packages/MLJ/w0rRg/src/tuning.jl:179


In [15]:
fitted_params(tuned_blended)

(best_model = KNNRidgeBlend{} @ 1…61,)

In [16]:
best_model = ans.best_model
@show best_model.knn_model.K;
@show best_model.ridge_model.lambda;

(best_model.knn_model).K = 2
(best_model.ridge_model).lambda = 2.3101297000831598


In [17]:
best = machine(best_model, X, y)
evaluate!(best, resampling=Holdout(fraction_train=0.7), measure=rmsl)

┌ Info: Evaluating using a holdout set. 
│ fraction_train=0.7 
│ shuffle=false 
│ measure=MLJ.rmsl 
│ operation=StatsBase.predict 
│ Resampling from all rows. 
└ @ MLJ /Users/anthony/.julia/packages/MLJ/w0rRg/src/resampling.jl:91


0.13067575064806702